# Model debugging

Debug file. Use `models/<versions>.py` to define the actual model architecture

In [1]:
import torch
import torch.nn as nn
from torchvision import models

In [91]:
%run models/v3.py

In [92]:
model = ResnetBasedModel()
_ = model.train(True)

In [93]:
n_samples = 4
image_channels = 3
image_width = 512
image_height = 512

image = torch.Tensor(n_samples, image_channels, image_height, image_width)
y, embeddings, activations = model(image)

print("Embedding: ", embeddings.size())
print("Activation: ", activations.size())
print("Output: ", y.size())

AttributeError: 'Tensor' object has no attribute 'retains_grad'

In [29]:
loss = torch.sum(y)
loss

tensor(27.6171, grad_fn=<SumBackward0>)

In [30]:
loss.backward()

In [31]:
for name, param in model.prediction.named_parameters():
    print(name, param.size())

0.weight torch.Size([14, 2048])
0.bias torch.Size([14])


In [32]:
linear = getattr(model.prediction, "0")

In [35]:
linear.weight.size()

torch.Size([14, 2048])

In [36]:
linear.weight.grad.size()

torch.Size([14, 2048])

In [42]:
pooling = getattr(model.global_pool, "0")
pooling

MaxPool2d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)

In [49]:
pooling.__dict__

{'_backend': <torch.nn.backends.thnn.THNNFunctionBackend at 0x7fee05188240>,
 '_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_parameters': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 'ceil_mode': False,
 'dilation': 1,
 'kernel_size': 16,
 'padding': 0,
 'return_indices': False,
 'stride': 16,
 'training': True}

In [70]:
model.model_ft.layer4

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2

In [71]:
last_bottleneck = getattr(model.model_ft.layer4, "2")
last_bottleneck

Bottleneck(
  (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
)

In [77]:
last_bottleneck.relu.register_hook

AttributeError: 'ReLU' object has no attribute 'register_hook'

In [69]:
getattr(last_bottleneck.downsample, "1").weight.register_hook()

<bound method Tensor.register_hook of Parameter containing:
tensor([0.2392, 0.2747, 0.2593,  ..., 0.1803, 0.2701, 0.2526])>

In [57]:
last_bottleneck.grad

AttributeError: 'Bottleneck' object has no attribute 'grad'